In [1]:
import sys
sys.path.append('..')

import login
login.connect()
import util
from schema import hheise_behav

Connecting hheise@130.60.53.47:3306


In [5]:
date = '2021-06-23'
mice = (81,82,83,85,86,87,89,90,91,92,93,94,95)

util.add_many_sessions(date, mice)

Inserted new session: {'username': 'hheise', 'mouse_id': 81, 'day': '2021-06-23', 'session_num': 1}
Inserted new session: {'username': 'hheise', 'mouse_id': 82, 'day': '2021-06-23', 'session_num': 1}
Inserted new session: {'username': 'hheise', 'mouse_id': 83, 'day': '2021-06-23', 'session_num': 1}
Inserted new session: {'username': 'hheise', 'mouse_id': 85, 'day': '2021-06-23', 'session_num': 1}
Inserted new session: {'username': 'hheise', 'mouse_id': 86, 'day': '2021-06-23', 'session_num': 1}
Inserted new session: {'username': 'hheise', 'mouse_id': 87, 'day': '2021-06-23', 'session_num': 1}
Inserted new session: {'username': 'hheise', 'mouse_id': 89, 'day': '2021-06-23', 'session_num': 1}
Inserted new session: {'username': 'hheise', 'mouse_id': 90, 'day': '2021-06-23', 'session_num': 1}
Inserted new session: {'username': 'hheise', 'mouse_id': 91, 'day': '2021-06-23', 'session_num': 1}
Inserted new session: {'username': 'hheise', 'mouse_id': 92, 'day': '2021-06-23', 'session_num': 1}


In [ ]:
hheise_behav.VRSession.populate('username="hheise"')